# Tutorial 6: Memory Systems in LangChain

In this tutorial, we'll explore memory systems in LangChain, which allow us to create more contextual and personalized interactions with language models.

## 1. Types of memory in LangChain

LangChain provides several types of memory systems:
- Buffer Memory
- Summary Memory
- Conversation Buffer Memory
- Conversation Summary Memory
- Vector Store-Backed Memory

Let's start by importing the necessary modules:

In [12]:
import os
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory,ConversationSummaryBufferMemory,ConversationBufferWindowMemory,CombinedMemory
from langchain_groq import ChatGroq
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

# Set up the Groq LLM
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"])

## 2. Implementing conversation memory

Let's implement a simple conversation chain with buffer memory:

In [31]:
# Create a ConversationBufferMemory
# buffer_memory = ConversationBufferMemory()
# summary_memory = ConversationSummaryMemory(llm=llm)
# memory2 = ConversationBufferWindowMemory(k=1)
# memory3 = CombinedMemory(memories=[summary_memory, buffer_memory])

 #Configure memories with unique variable names
buffer_memory = ConversationBufferMemory(memory_key="buffer_history")
summary_memory = ConversationSummaryMemory(llm=llm, memory_key="summary_history")
window_memory = ConversationBufferWindowMemory(k=1, memory_key="window_history")

combined_memory = CombinedMemory(
    memories=[summary_memory, buffer_memory, window_memory]
)
    
# Define a prompt template
template = """
You are a creative writer. Write a short story (about 100 words) based on the following prompt:

Prompt: {prompt}

Story:
Summary:
{summary_history}
Buffer:
{buffer_history}
Window:
{window_history}
"""

prompt = PromptTemplate(template=template, memories=[buffer_memory, summary_memory, window_memory])
# Create a ConversationChain with buffer memory
# Create a chain
story_chain = prompt | llm

# Start a conversation
print(story_chain.invoke("Hi, my name is Alice."))
print(story_chain.invoke("What's my name?"))
print(story_chain.invoke("Tell me a fun fact about my name."))

/home/domenico/miniconda3/lib/python3.12/site-packages/langchain/memory/combined.py:38: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x79ce23e36000>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x79ce23d4b8c0>, groq_api_key=SecretStr('**********')) memory_key='summary_history'
  warnings.warn(
/home/domenico/miniconda3/lib/python3.12/site-packages/langchain/memory/combined.py:38: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on memory_key='buffer_history'
  warnings.warn(
/home/domenico/miniconda3/lib/python3.12/site-packages/langchain/memory/combined.py:38: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on memory_key='window_history' k=1
  warnings.warn(


TypeError: Expected mapping type as input to PromptTemplate. Received <class 'str'>.

In [19]:
import os
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory, 
    ConversationBufferWindowMemory,
    CombinedMemory
)
from langchain_groq import ChatGroq
from langchain.chains import ConversationChain

def create_conversation_chain(api_key):
    llm = ChatGroq(api_key=api_key)
    
    # Configure memories with unique variable names
    buffer_memory = ConversationBufferMemory(memory_key="buffer_history")
    summary_memory = ConversationSummaryMemory(llm=llm, memory_key="summary_history")
    window_memory = ConversationBufferWindowMemory(k=1, memory_key="window_history")
    
    combined_memory = CombinedMemory(
        memories=[summary_memory, buffer_memory, window_memory]
    )
    
    return ConversationChain(
        llm=llm,
        memory=combined_memory,
        verbose=True
    )

def simulate_conversation(conversation):
    inputs = [
        "Hi, my name is Alice.",
        "What's my name?",
        "Tell me a fun fact about my name.",
        "What did I ask you earlier?"
    ]
    
    for input_text in inputs:
        response = conversation.predict(input=input_text)
        print(f"Input: {input_text}")
        print(f"Response: {response}\n")

if __name__ == "__main__":
    api_key = os.environ["GROQ_API_KEY"]
    conversation = create_conversation_chain(api_key)
    simulate_conversation(conversation)

/home/domenico/miniconda3/lib/python3.12/site-packages/langchain/memory/combined.py:38: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x79ce22c3b4a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x79ce22c395b0>, groq_api_key=SecretStr('**********')) memory_key='summary_history'
  warnings.warn(
/home/domenico/miniconda3/lib/python3.12/site-packages/langchain/memory/combined.py:38: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on memory_key='buffer_history'
  warnings.warn(
/home/domenico/miniconda3/lib/python3.12/site-packages/langchain/memory/combined.py:38: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on memory_key='window_history' k=1
  warnings.warn(


ValidationError: 1 validation error for ConversationChain
__root__
  Got unexpected prompt input variables. The prompt expects ['history', 'input'], but got ['summary_history', 'buffer_history', 'window_history'] as inputs from memory, and input as the normal input key. (type=value_error)

## 3. Creating a chatbot with long-term memory

Now, let's create a chatbot with summary memory for longer conversations:

In [3]:
# Create a ConversationSummaryMemory
summary_memory = ConversationSummaryMemory(llm=llm)

# Create a ConversationChain with summary memory
chatbot = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=True
)

# Simulate a longer conversation
responses = [
    chatbot.predict(input="Hi, I'm Bob. I'm planning a trip to Paris."),
    chatbot.predict(input="What are some must-visit attractions?"),
    chatbot.predict(input="I'm also interested in trying local cuisine. Any recommendations?"),
    chatbot.predict(input="Lastly, what's the best time of year to visit Paris?")
]

for response in responses:
    print(response)

# Check the summarized memory
print("\nSummarized Memory:")
print(summary_memory.load_memory_variables({})['history'])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm Bob. I'm planning a trip to Paris.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human, introducing himself as Bob, shares his plan to visit Paris with the AI assistant. The AI, welcoming Bob, expresses enthusiasm about Paris's rich history, stunning architecture, museums, and romantic ambiance. The AI then asks for more details about Bob's travel dates a

## 4. Advanced memory techniques Knowledge Graphs Memory

Let's explore an advanced technique: Knowledge Graphs Memory. This technique allows us to store information in a graph structure, which can be queried and updated.

In [21]:
from langchain.chains.conversation.memory import ConversationKGMemory

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)

# Create a conversation chain with combined memory and custom prompt
advanced_chatbot = ConversationChain(
    llm=llm,
    memory=ConversationKGMemory(llm=llm),
    prompt=prompt,
    verbose=True
)
# # Simulate a conversation with the advanced chatbot
responses = [
    advanced_chatbot.predict(input="Hi, I'm Charlie. I love science fiction books."),
    advanced_chatbot.predict(input="Can you recommend a classic sci-fi novel?"),
    advanced_chatbot.predict(input="Thanks! Now, tell me about a recent sci-fi movie."),
    advanced_chatbot.predict(input="What was my name again? And what genre do I like?")
]

for response in responses:
    print(response)
    



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Hi, I'm Charlie. I love science fiction books.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Can you recommend a classic 

In [22]:
print(advanced_chatbot.memory.kg.get_triples())

[('Charlie', 'human', 'is a'), ('Charlie', 'science fiction books', 'loves'), ('AI', 'a classic sci-fi novel', 'can recommend')]


## Conclusion

In this tutorial, we've explored various memory systems in LangChain, from simple buffer memory to more advanced techniques like combined memory with custom prompts. These memory systems allow you to create more contextual and personalized interactions in your language model applications.